In [5]:
from sqlalchemy import create_engine, text
import pandas as pd
import numpy as np
import psycopg2
from qiskit_aer import Aer
from qiskit.circuit.library import RealAmplitudes
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import SparsePauliOp

# ✅ DATABASE CONNECTION SETTINGS
DB_HOST = "192.168.0.34"
DB_NAME = "rag_db"
DB_USER = "user"
DB_PASS = "password"
DB_PORT = "5432"
DATABASE_URL = f"postgresql+psycopg2://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(DATABASE_URL)

def run_quantum_session():
    """Runs a quantum process to generate business rules dynamically."""

    num_qubits = 3  # Encoding FICO, Utilization, and Income
    ansatz = RealAmplitudes(num_qubits, reps=2)

    # ✅ Define Quantum Cost Function (Hamiltonian)
    H = SparsePauliOp(
        ["ZII", "IZI", "IIZ"],  # Operators affecting different attributes
        coeffs=np.array([0.4, -0.3, 0.3])  # Adjusting impact of FICO, Utilization, and Income
    )

    # ✅ Optimizer and Quantum Execution
    optimizer = COBYLA(maxiter=100)
    backend = Aer.get_backend('aer_simulator')

    from qiskit.primitives import Estimator
    vqe = VQE(Estimator(), ansatz, optimizer=optimizer)

    # ✅ Compute Optimal Value
    result = vqe.compute_minimum_eigenvalue(H)
    optimal_value = np.clip(result.optimal_value, 1.0, 1.25)  # Ensure within range

    return optimal_value  # Quantum-optimized multiplier for credit decision



# ✅ GENERATE QUANTUM BUSINESS RULES
def generate_business_rules():
    """Maps quantum output to business rules."""
    quantum_multiplier = run_quantum_session()  # Run quantum optimization

    # ✅ Assigning Business Rules Based on Quantum Decision
    rules = {
        "Approve Credit Increase": quantum_multiplier,
        "Approve Small Increase": 1.10,  # Fixed small increase
        "Deny Increase": 1.00,  # No change
        "Flag for Manual Review": "Manual Review"
    }

    return rules

# ✅ STORE RULES IN POSTGRESQL DATABASE
def store_rules_in_db(rules):
    """Saves the generated business rules in the database."""
    conn = engine.connect()
    trans = conn.begin()
    
    try:
        for rule, multiplier in rules.items():
            query = text("""
            INSERT INTO quantum_business_rules (decision_rule, credit_multiplier, last_updated)
            VALUES (:decision_rule, :credit_multiplier, NOW())
            ON CONFLICT (decision_rule) DO UPDATE 
            SET credit_multiplier = EXCLUDED.credit_multiplier, last_updated = NOW();
            """)
            conn.execute(query, {"decision_rule": rule, "credit_multiplier": str(multiplier)})
        
        trans.commit()
    except Exception as e:
        trans.rollback()
        print(f"Error storing rules in database: {e}")
    finally:
        conn.close()

# ✅ APPLY BUSINESS RULES DYNAMICALLY
def execute_business_rule(credit_limit, rule_name):
    """Applies the business rule to adjust credit limit."""
    conn = engine.connect()
    
    query = text("""
    SELECT credit_multiplier FROM quantum_business_rules WHERE decision_rule = :rule_name;
    """)
    
    result = conn.execute(query, {"rule_name": rule_name}).fetchone()
    conn.close()

    if result:
        multiplier = result[0]
        if multiplier == "Manual Review":
            return "Manual Review Required"
        return float(credit_limit) * float(multiplier)
    
    return "Error"

# ✅ MAIN EXECUTION (DAILY PROCESS)
if __name__ == "__main__":
    print("🔹 Running Quantum Session to Generate Business Rules...")
    business_rules = generate_business_rules()
    
    print("🔹 Storing Rules in Database...")
    store_rules_in_db(business_rules)

    print("✅ Quantum Business Rules Updated Successfully!")

    # ✅ Example Usage
    test_credit_limit = 5000
    updated_limit = execute_business_rule(test_credit_limit, "Approve Credit Increase")
    print(f"Updated Credit Limit: {updated_limit}")


🔹 Running Quantum Session to Generate Business Rules...


/tmp/ipykernel_2858650/1274131995.py:37: DeprecationWarning: The class ``qiskit.primitives.estimator.Estimator`` is deprecated as of qiskit 1.2. It will be removed no earlier than 3 months after the release date. All implementations of the `BaseEstimatorV1` interface have been deprecated in favor of their V2 counterparts. The V2 alternative for the `Estimator` class is `StatevectorEstimator`.
  vqe = VQE(Estimator(), ansatz, optimizer=optimizer)


🔹 Storing Rules in Database...
✅ Quantum Business Rules Updated Successfully!
Updated Credit Limit: 5000.0
